In [ ]:
import numpy as np
import matplotlib.pylab as plt
from operator import itemgetter
import pandas as pd

In [ ]:
natl = np.loadtxt('../National_Ratings_by_Game.csv',dtype=str,delimiter=',')
teams,tvs = np.loadtxt('../DMA_Households.csv',dtype=str,delimiter=',',skiprows=1,unpack=True)

In [ ]:
data_dict = {}
data_dict['Team'] = []
data_dict['Nat\'l Average'] = []
data_dict['Avg Rating Same Night'] = []
data_dict['Team Average Rating'] = []
data_dict['Season'] = []
data_dict['Delta Avg'] = []
xticks = ['2014-15','2015-16','2016-17','2017-18']


for team in teams:
    
    gs = team
    home = natl[:,7]==gs
    away = natl[:,8]==gs
    played = np.logical_or(away,home)
    all_dates = natl[:,3] # haystack
    dates = natl[played][:,3] # needles
    st = set(dates)
    date_indicies = ([i for i, e in enumerate(all_dates) if e in st])
    other_days = ([i for i, e in enumerate(all_dates) if e not in st])

    # list of dates

    day_gsw_televised = natl[date_indicies]
    day_not_televised = natl[other_days]

#     print("National Ratings season to season: \n \n ")
    lGSW = []
    lAll = []
    lTEAM = []
    lTEAM_without = []
    lOther_Days = []
    

    for season in xticks:

        all_year = natl[:,0]==season
        all_NoNBA = natl[:,2]!='NBA TV'
        ind1 = np.logical_and(all_NoNBA,all_year)
        all_r = natl[ind1][:,9]
        all_r = all_r.astype(np.float)


        year = day_gsw_televised[:,0]==season
        noNBATV = day_gsw_televised[:,2]!='NBA TV'
        ind = np.logical_and(year,noNBATV)
        no_GSWh = day_gsw_televised[:,7]!=team
        no_GSWa = day_gsw_televised[:,8]!=team
        no_GSW = np.logical_and(no_GSWh,no_GSWa)
        GSW = ~no_GSW
        ind4 = np.logical_and(ind,GSW)
        ind2 = np.logical_and(no_GSW,ind)

        ratings = day_gsw_televised[ind][:,9]
        ratings = ratings.astype(np.float)
        ratings_noGSW = day_gsw_televised[ind2][:,9]
        ratings_noGSW = ratings_noGSW.astype(np.float)	
        ratings_onlyGSW = day_gsw_televised[ind4][:,9]
        ratings_onlyGSW = ratings_onlyGSW.astype(np.float)

        year = day_not_televised[:,0]==season
        noNBA = day_not_televised[:,2]!='NBA TV'
        ind = np.logical_and(noNBA,year)
        r2 = day_not_televised[ind][:,9]
        r2 = r2.astype(np.float)	

        All = np.nanmean(all_r)
        TEAM = np.nanmean(ratings)
        TEAM_without = np.nanmean(ratings_noGSW)
        Other_Days = np.nanmean(r2)

        lAll.append(All)
        lTEAM.append(TEAM)
        lTEAM_without.append(TEAM_without)
        lOther_Days.append(Other_Days)	
        lGSW.append(np.nanmean(ratings_onlyGSW))
        
        
    for i in range(4):
        data_dict['Season'].append(xticks[i])
        data_dict['Team'].append(team)
        data_dict['Nat\'l Average'].append(lAll[i])
        data_dict['Avg Rating Same Night'].append(lTEAM[i])
        data_dict['Team Average Rating'].append(lGSW[i])
        data_dict['Delta Avg'].append(lTEAM[i]-lAll[i])

In [ ]:
dataFrame = pd.DataFrame(data=data_dict)
dataFrame = dataFrame[["Season","Team","Team Average Rating","Avg Rating Same Night", "Nat'l Average", "Delta Avg"]]
df = dataFrame.sort_values(["Season","Team"])
df.to_csv('../natl_ratings.csv')

In [ ]:
df